In [35]:
from gensim.models import KeyedVectors,Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stopWords = set(stopwords.words("english"))
import nltk
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\65835\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [60]:
filename = r'C:\Users\65835\Desktop\projects\googlenewsvectorsnegative300\GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [97]:
# To tokenize the string into words

def tokenizing(ss1,ss2):
    sentence1 = word_tokenize(ss1)
    sentence2 = word_tokenize(ss2)
    return sentence1,sentence2
    


In [288]:
# To clean the data : remove non alphanumeric characters

def cleaning(sentence1,sentence2):
    cleaned_ss1=[]
    for i in sentence1:
        if (i not in stopWords) and (i.isalnum()) :
            cleaned_ss1.append(i)

    cleaned_ss2=[]
    for i in sentence2:
        if i not in stopWords and i.isalnum():
            cleaned_ss2.append(i)
    return cleaned_ss1,cleaned_ss2
    

In [302]:
# the first function with a user defined formula to check if two sentences are related or similar

def check_spam(cleaned_ss1,cleaned_ss2):
    count = 0
    for i in cleaned_ss1:
        for j in cleaned_ss2:
            try:
                if model.similarity(i,j)>=0.3 :
                    count+=1
            except:
                pass
    if count>=(len(cleaned_ss1)*len(cleaned_ss2))**0.5:
        spam = False
    else :
        spam = True
    return spam



In [303]:
# Function that returns the norm(magnitude) of the vector

def magnitude(vector):
    sum_of_squares = 0
    k=0
    for num in vector:
        sum_of_squares += (num*num)
        k+=1
    result = sum_of_squares**0.5
    return result

In [304]:
# Function that returns the dot product of two vectors

def dot_product(u,v):
    dot = 0
    k=0
    for num in v:
        dot+=u[k]*v[k]
        k=k+1
    return dot

In [305]:
# Second function to check if two sentences are similar

def double_check(cleaned_ss1,cleaned_ss2):
    tolerance = 0.3
    v=[0]
    for i in cleaned_ss1:
        try:
            v=v+model[i]
        except:
            pass
    u=[0]
    for i in cleaned_ss2:
        try:
            u=u+model[i]
        except:
            pass
    
    
    if(magnitude(u)==0 or magnitude(v)==0):
        spam = True
        cosine = 0
    else :
        cosine=dot_product(u,v)/(magnitude(v)*magnitude(u))
        if(cosine>tolerance):
            spam = False
        else:
            spam = True
    return spam
    

In [306]:
# Third function to check if two sentences are similar or not

def triple_check(cleaned_ss1,cleaned_ss2):
    tolerance = 1.5
    v=[0]
    for i in cleaned_ss1:
        try:
            v=v+model[i]
        except:
            pass
    u=[0]
    for i in cleaned_ss2:
        try:
            u=u+model[i]
        except:
            pass
    diff = magnitude(u)-magnitude(v)
    
    if(-tolerance<=diff<=tolerance):
        spam = False
    else:
        spam = True
    return spam

In [309]:

# string values of headline and summary must be taken from a column from the main dataframe using a loop(must put whole code in loop after both columns have been extracted)

headline = "ASAP Ferg And Remy Ma Rep The East Coast On New Song "
headline = headline.lower()
summary = "ASAP Ferg is repping for the East Coa on his new song, and brings fellow NYC rapper Remy Ma along for the ride."
summary = summary.lower()


headline,summary = tokenizing(headline , summary)
headline,summary = cleaning(headline , summary)


check_1 = check_spam(headline , summary)
check_2 = double_check(headline , summary)
check_3 = triple_check(headline , summary)

# If any 2 out of the 3 check variables are True , then the news article is click-bait.
    

Spam : False
Spam : False
Spam : True
